In [78]:
import os
import re
from configparser import ConfigParser, NoOptionError

import numpy as np
import pandas as pd
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy.tools import ToolClient
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import mapper, sessionmaker

In [79]:
galaxy_url = 'http://127.0.0.1:8080'
api_key = os.getenv('GALAXY_API_KEY')
gi = GalaxyInstance(url=galaxy_url, key=api_key)

### Connect to the database

In [80]:
galaxy_path = os.getenv('GALAXY_DIRECTORY')
config_file = os.path.join(galaxy_path, 'config/galaxy.ini.sample')
db_path = ''

if os.path.isfile(config_file):
    config = ConfigParser()
    config.read_file(open(config_file))
    
    try:
        # Get database path     
        db_path = config.get('app:main', 'database_connection')
#         db_path = 'postgresql://galaxy@localhost:5432/galaxy'  # temporary
            
    except NoOptionError:
        # Default
        db_path = 'sqlite:///' + os.path.join(galaxy_path, 'database/universe.sqlite')

engine = create_engine(db_path)

## Connections

In [81]:
query_get_connections = '''
SELECT DISTINCT ws.id, ws.workflow_id, ws.tool_id,
    ws.tool_version, ws.label, wsa.annotation,
    wsc.output_step_id AS in_port, wsc.input_step_id AS out_port

FROM workflow_step as ws, stored_workflow as sw

LEFT JOIN workflow_step_connection AS wsc
    ON ws.id = wsc.output_step_id

LEFT JOIN workflow_step_annotation_association as wsa
    ON ws.id = wsa.workflow_step_id

WHERE ws.workflow_id = sw.latest_workflow_id AND ws.type != 'data_input'
'''
connections = pd.read_sql(query_get_connections, con=engine)
connections[['in_port', 'out_port']] = connections[['in_port', 'out_port']].fillna(0.0).applymap(np.int64)

# def get_tool_ids(port):
#     for port_id in connections[port]:
#         tool_id = connections.tool_id[connections.id == port_id]
#         yield tool_id.values[0] if not tool_id.empty else None

# in_port_ids = pd.Series(list(get_tool_ids('in_port')), name='in_port_tool')
# out_port_ids = pd.Series(list(get_tool_ids('out_port')), name='out_port_tool')
# connections = pd.concat([connections, in_port_ids, out_port_ids], axis=1)
# connections = connections[(connections.in_port != 0) & (connections.out_port != 0)]

connections

,id,workflow_id,tool_id,tool_version,label,annotation,in_port,out_port
0,18,6,Cut1,1.0.2,cut 1,cut 1 annot,18,20
1,19,6,Cut1,1.0.2,cut 2,cut 2 annot,19,20
2,20,6,cat1,1.0.0,output,output annot,0,0
3,23,8,Cut1,1.0.2,,None,23,26
4,24,8,mergeCols1,1.0.1,,None,24,25
5,25,8,Show beginning1,1.0.0,,None,25,26
6,26,8,cat1,1.0.0,,None,26,27
7,27,8,trimmer,0.0.1,,None,0,0
8,29,10,gene2exon1,1.0.0,,None,29,30
9,30,10,bed2gff1,2.0.0,,None,0,0


## Tool information

In [82]:
query_get_tools = '''
SELECT DISTINCT ws.tool_id AS id, ws.tool_version AS version
FROM workflow_step as ws, stored_workflow as sw
WHERE ws.tool_id IS NOT NULL AND ws.workflow_id = sw.latest_workflow_id
'''
workflow_tools = pd.read_sql(query_get_tools, con=engine)
workflow_tools['flag'] = 1

tc = ToolClient(gi)
all_tools = pd.DataFrame.from_dict(tc.get_tools())

tools = pd.merge(all_tools, workflow_tools, on=['id', 'version'], how='left')
tools = tools[pd.notnull(tools['flag'])].drop(['flag'], axis=1)

tools

,description,edam_operations,edam_topics,form_style,id,labels,link,min_width,model_class,name,panel_section_id,panel_section_name,target,tool_shed_repository,version
9,side by side on a specified field,[],[],regular,join1,[],/tool_runner?tool_id=join1,-1,Tool,Join two Datasets,group,"Join, Subtract and Group",galaxy_main,NaN,2.0.2
41,leading or trailing characters,[],[],regular,trimmer,[],/tool_runner?tool_id=trimmer,-1,Tool,Trim,textutil,Text Manipulation,galaxy_main,NaN,0.0.1
79,together,[],[],regular,mergeCols1,[],/tool_runner?tool_id=mergeCols1,-1,Tool,Merge Columns,textutil,Text Manipulation,galaxy_main,NaN,1.0.1
86,delimiters to TAB,[],[],regular,Convert characters1,[],/tool_runner?tool_id=Convert+characters1,-1,Tool,Convert,textutil,Text Manipulation,galaxy_main,NaN,1.0.0
97,expander,[],[],regular,gene2exon1,[],/tool_runner?tool_id=gene2exon1,-1,Tool,Gene BED To Exon/Intron/Codon BED,features,Extract Features,galaxy_main,NaN,1.0.0
98,columns from a table,[],[],regular,Cut1,[],/tool_runner?tool_id=Cut1,-1,Tool,Cut,textutil,Text Manipulation,galaxy_main,NaN,1.0.2
118,tail-to-head,[],[],regular,cat1,[],/tool_runner?tool_id=cat1,-1,Tool,Concatenate datasets,textutil,Text Manipulation,galaxy_main,NaN,1.0.0
128,converter,[operation_3434],[],regular,bed2gff1,[],/tool_runner?tool_id=bed2gff1,-1,Tool,BED-to-GFF,convert,Convert Formats,galaxy_main,NaN,2.0.0
137,lines from a dataset,[],[],regular,Show beginning1,[],/tool_runner?tool_id=Show+beginning1,-1,Tool,Select first,textutil,Text Manipulation,galaxy_main,NaN,1.0.0


# ORM

In [83]:
class WorkflowStep:
    pass

class StoredWorkflow:
    pass

class WorkflowStepConnection:
    pass

class WorkflowStepAnnotationAssociation:
    pass

# Create SQLAlchemy Tables
metadata = MetaData(engine)
ws = Table('workflow_step', metadata, autoload=True)
sw = Table('stored_workflow', metadata, autoload=True)
wsc = Table('workflow_step_connection', metadata, autoload=True)
wsaa = Table('workflow_step_annotation_association', metadata, autoload=True)

mapper(WorkflowStep, ws)
mapper(StoredWorkflow, sw)
mapper(WorkflowStepConnection, wsc)
mapper(WorkflowStepAnnotationAssociation, wsaa)

# Create Session object
Session = sessionmaker(bind=engine)
session = Session()

In [144]:
connections_query = session \
    .query(WorkflowStep.id, WorkflowStep.workflow_id,
           WorkflowStep.type, WorkflowStep.tool_id, 
           WorkflowStep.tool_version, WorkflowStep.label,
           WorkflowStepAnnotationAssociation.annotation,
           WorkflowStepConnection.output_step_id.label('in_port'),
           WorkflowStepConnection.input_step_id.label('out_port')) \
    .join(StoredWorkflow, WorkflowStep.workflow_id==\
          StoredWorkflow.latest_workflow_id) \
    .outerjoin(WorkflowStepConnection, WorkflowStep.id==\
               WorkflowStepConnection.output_step_id) \
    .outerjoin(WorkflowStepAnnotationAssociation, WorkflowStep.id==\
               WorkflowStepAnnotationAssociation.workflow_step_id) \
    .distinct()

connections = pd.read_sql(connections_query.statement, session.bind)
connections = connections.where(pd.notnull(connections), None)  # replace all NaN with None

connections['in_port_tool'] = connections.tool_id.fillna(connections.type)
connections['out_port_tool'] = [connections.tool_id[connections.id == id].values[0] 
                                if id else None for id in connections.out_port]
connections

,id,workflow_id,type,tool_id,tool_version,label,annotation,in_port,out_port,in_port_tool,out_port_tool
0,16,6,data_input,None,None,inp 1,inp 1 annot,16,18,data_input,Cut1
1,17,6,data_input,None,None,inp 2,inp 2 annot,17,19,data_input,Cut1
2,18,6,tool,Cut1,1.0.2,cut 1,cut 1 annot,18,20,Cut1,cat1
3,19,6,tool,Cut1,1.0.2,cut 2,cut 2 annot,19,20,Cut1,cat1
4,20,6,tool,cat1,1.0.0,output,output annot,None,None,cat1,None
5,21,7,data_input,None,None,None,None,None,None,data_input,None
6,22,8,data_input,None,None,None,None,22,23,data_input,Cut1
7,22,8,data_input,None,None,None,None,22,24,data_input,mergeCols1
8,23,8,tool,Cut1,1.0.2,,None,23,26,Cut1,cat1
9,24,8,tool,mergeCols1,1.0.1,,None,24,25,mergeCols1,Show beginning1


In [145]:
tools_query = session \
    .query(WorkflowStep.tool_id.label('id'),
           WorkflowStep.tool_version.label('version')) \
    .join(StoredWorkflow, WorkflowStep.workflow_id==\
          StoredWorkflow.latest_workflow_id) \
    .distinct().filter(WorkflowStep.tool_id.isnot(None))

workflow_tools = pd.read_sql(tools_query.statement, session.bind)
workflow_tools['flag'] = 1

tc = ToolClient(gi)
all_tools = pd.DataFrame.from_dict(tc.get_tools())

tools = pd.merge(all_tools, workflow_tools, on=['id', 'version'], how='left')
tools = tools[pd.notnull(tools['flag'])].drop(['flag'], axis=1)

tools

,description,edam_operations,edam_topics,form_style,id,labels,link,min_width,model_class,name,panel_section_id,panel_section_name,target,tool_shed_repository,version
9,side by side on a specified field,[],[],regular,join1,[],/tool_runner?tool_id=join1,-1,Tool,Join two Datasets,group,"Join, Subtract and Group",galaxy_main,NaN,2.0.2
41,leading or trailing characters,[],[],regular,trimmer,[],/tool_runner?tool_id=trimmer,-1,Tool,Trim,textutil,Text Manipulation,galaxy_main,NaN,0.0.1
79,together,[],[],regular,mergeCols1,[],/tool_runner?tool_id=mergeCols1,-1,Tool,Merge Columns,textutil,Text Manipulation,galaxy_main,NaN,1.0.1
86,delimiters to TAB,[],[],regular,Convert characters1,[],/tool_runner?tool_id=Convert+characters1,-1,Tool,Convert,textutil,Text Manipulation,galaxy_main,NaN,1.0.0
97,expander,[],[],regular,gene2exon1,[],/tool_runner?tool_id=gene2exon1,-1,Tool,Gene BED To Exon/Intron/Codon BED,features,Extract Features,galaxy_main,NaN,1.0.0
98,columns from a table,[],[],regular,Cut1,[],/tool_runner?tool_id=Cut1,-1,Tool,Cut,textutil,Text Manipulation,galaxy_main,NaN,1.0.2
118,tail-to-head,[],[],regular,cat1,[],/tool_runner?tool_id=cat1,-1,Tool,Concatenate datasets,textutil,Text Manipulation,galaxy_main,NaN,1.0.0
128,converter,[operation_3434],[],regular,bed2gff1,[],/tool_runner?tool_id=bed2gff1,-1,Tool,BED-to-GFF,convert,Convert Formats,galaxy_main,NaN,2.0.0
137,lines from a dataset,[],[],regular,Show beginning1,[],/tool_runner?tool_id=Show+beginning1,-1,Tool,Select first,textutil,Text Manipulation,galaxy_main,NaN,1.0.0


In [166]:
counts = connections[['in_port_tool', 'out_port_tool', 'id']].groupby(['in_port_tool', 'out_port_tool'])

counts.count().rename(columns={'id': 'count'})

count
in_port_tool          out_port_tool             
Cut1                  Convert characters1      1
                      cat1                     3
Show beginning1       cat1                     1
cat1                  trimmer                  1
data_collection_input Cut1                     1
data_input            Cut1                     3
                      gene2exon1               1
                      join1                    2
                      mergeCols1               1
gene2exon1            bed2gff1                 1
join1                 Cut1                     1
mergeCols1            Show beginning1          1

In [162]:
suggestions = counts.size()['data_input'].sort_values(ascending=False)[:5].axes[0].tolist()

suggestions

['Cut1', 'join1', 'mergeCols1', 'gene2exon1']